In [ ]:
from langchain.chat_models import ChatOpenAI # 하는 역할 :챗 모델 불러오기
from langchain.prompts import ChatPromptTemplate # 하는 역할 :챗 프롬프트 템플릿 불러오기
from langchain.schema import BaseOutputParser # 하는 역할 :출력 파서의 기본 클래스 불러오기
chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

# 하는 역할 : 사용자 정의 출력 파서 생성
class CustomParser(BaseOutputParser): 
    def parse(self, text: str) -> str:
        return text.split("\n")[0]

# 하는 역할 : 요약 프롬프트 및 체인 생성    
summary_prompt = ChatPromptTemplate.from_messages([
    ("system", "다음 질문을 한 문장으로 요약하세요."),
    ("human", "{question}")
])

# 하는 역할 : 요약 체인 생성
summary_chain = summary_prompt | chat | CustomParser()
# 하는 역할 : 요약을 딕셔너리로 변환하는 람다 함수 생성
summary_to_dict = lambda x: {"summary": x}
# 하는 역할 : 답변 프롬프트 및 체인 생성
answer_prompt = ChatPromptTemplate.from_messages([
    ("system", "다음 질문에 간단히 답하세요."),
    ("human", "{summary}")
])
# 하는 역할 : 답변 체인 생성
answer_chain = answer_prompt | chat | CustomParser()
# 하는 역할 : 전체 체인 생성
chain = summary_chain | summary_to_dict | answer_chain
# 하는 역할 : 체인 실행 및 결과 출력
result = chain.invoke({
    "question": "태양계에는 왜 행성이 8개인가요?"
})

print(result)

In [ ]:
from langchain.prompts.few_shot import FewShotPromptTemplate # 하는 역할 :몇 가지 예제를 포함한 프롬프트 생성
from langchain.prompts.prompt import PromptTemplate # 하는 역할 : 프롬프트 템플릿 생성
from langchain.prompts.example_selector import LengthBasedExampleSelector # 하는 역할 : 예제 선택 시 길이를 기준으로 선택
from langchain.prompts.pipeline import PipelinePromptTemplate  # 하는 역할 : 여러 프롬프트를 순차적으로 연결하여 복잡한 프롬프트를 생성
from langchain.globals import set_llm_cache # 하는 역할 : LLM 캐시 설정
from langchain.globals import set_debug # 하는 역할 : 디버그 모드 설정
from langchain.cache import InMemoryCache # 하는 역할 : 메모리 기반 캐시 구현
from langchain.cache import SQLiteCache # 하는 역할 : SQLite 기반 캐시 구현

# 하는 역할 : LLM 캐시를 메모리 기반 캐시로 설정
set_llm_cache(InMemoryCache())

# 하는 역할 : 디버그 모드 활성화
set_debug(True)


chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

chat.predict("자바 실무 개발에 대해 알려줘.")

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.schema import BaseOutputParser

chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

# 하는 역할 : 사용자 정의 출력 파서 생성
class CustomParser(BaseOutputParser): 
    def parse(self, text: str) -> str:
        return text.split("\n")[0]


# 하는 역할 :출력 파서 인스턴스 생성
examples = [
    {
        "input": "Inception",
        "output": (
            "Title: Inception\n"
            "Director: Christopher Nolan\n"
            "Cast: Leonardo DiCaprio, Joseph Gordon-Levitt, Ellen Page\n"
            "Budget: $160 million\n"
            "Box Office: $836.8 million\n"
            "Genre: Science Fiction, Thriller\n"
            "Synopsis: A skilled thief enters people's dreams to steal secrets, "
            "but is offered a chance to erase his past crimes by planting an idea instead."
        )
    }
]

# 하는 역할 : 예제 프롬프트 템플릿 생성
example_prompt = ChatPromptTemplate.from_messages([
    ("human", "Movie: {input}"),
    ("ai", "{output}")
])

# 하는 역할 : 몇 가지 예제를 포함한 프롬프트 템플릿 생성
few_shot_prompt = FewShotChatMessagePromptTemplate(
    examples=examples,
    example_prompt=example_prompt
)

# 하는 역할 :최종 프롬프트 템플릿 생성
final_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a movie information assistant. Always use the exact same format."),
    few_shot_prompt,
    ("human", "Movie: {movie_name}")
])

# 하는 역할 :최종 체인 생성
chain = final_prompt | chat | CustomParser()

# 하는 역할 :체인 실행 및 결과 출력
result = chain.invoke({
    "movie_name": "Interstellar"
})

print(result)

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate,FewShotChatMessagePromptTemplate, MessagesPlaceholder ,ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.memory import ConversationSummaryBufferMemory

# 하는 역할 :챗 모델 인스턴스 생성
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

# 하는 역할 :대화 요약 버퍼 메모리 인스턴스 생성
memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=50, return_messages=True)

def load_history(_):
    return memory.load_memory_variables({})["history"]

# 하는 역할 :이모지 표현을 위한 예제들 정의
examples = [
    {"question": "대부", "answer": "👨‍👨‍👦🔫🍝"},
    {"question": "반지의 제왕", "answer": "🦸‍♂️🧙‍♂️💍"}
]

# 하는 역할 :예제 프롬프트 템플릿 생성
example_prompt = ChatPromptTemplate.from_messages([
    ("human", "질문: {question}"),
    ("ai", "답변: {answer}")
])

# 하는 역할 :몇 가지 예제를 포함한 프롬프트 템플릿 생성
prompt = FewShotChatMessagePromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
)

# 하는 역할 :최종 프롬프트 템플릿 생성
final_prompt = ChatPromptTemplate.from_messages([
    ("system", "항상 정확히 세 개의 이모지로만 답변하세요."),
    MessagesPlaceholder(variable_name="history"),
    prompt,
    ("human", "{question}"),
])

# 하는 역할 :전체 체인 생성
chain = RunnablePassthrough.assign(history=load_history) | final_prompt | llm

question = "해리포터"

# 하는 역할 :체인 실행 및 결과 출력
result = chain.invoke({
    "question": question
})
# 하는 역할 :결과 출력
print(result.content)

memory.save_context(
    {"input": question},
    {"output": result.content}
)

답변: ⚡️🧙‍♂️🔮
